extract_features.py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Import packages for image processing
from PIL import Image   
from skimage import morphology  # For morphological operations
from skimage.measure import find_contours, label    # For finding contours
from skimage.morphology import binary_closing   # For closing operation
from skimage.color import rgb2lab, rgb2gray   # For color space conversion
from skimage.transform import rotate, resize    # For rotating and resizing images
from skimage.measure import regionprops   # For region properties
from skimage.feature import graycomatrix, graycoprops   # For GLCM
from skimage.util import img_as_ubyte   # For scaling images to 8-bit
from scipy.ndimage import center_of_mass, rotate    # For calculating center of mass and rotating images
from skimage.util import img_as_ubyte  # For scaling images to 8-bit


#-------------------
# Help functions
#------------------



#Main function to extract features from an image, that calls other functions    
def extract_features(image_obj, mask_obj):
    
    # Resize the image and the mask to the correct size
    image = image_obj.resize((512, 512), resample=Image.BILINEAR)
    mask = mask_obj.resize((512, 512), resample=Image.BILINEAR)
    
    # Apply mask to image
    image = np.array(image)
    # Convert PIL Image to numpy array and ensure it's in the correct format for mask
    mask = np.array(mask).astype(np.uint8)
    # Apply mask to image
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    if masked_image.shape[2] == 4:  # If image has 4 channels
        masked_image = cv2.cvtColor(masked_image, cv2.COLOR_RGBA2RGB)
    
    
    # other formats
    inverted_mask = cv2.bitwise_not(np.array(mask))
    masked_image_arr = np.array(masked_image)
    gray_image = cv2.cvtColor(masked_image_arr, cv2.COLOR_RGB2GRAY)
    # Convert NumPy array back to PIL Image
    inverted_mask = Image.fromarray(inverted_mask)  
    
    # Calculate asymmetry scores
    asymmetry_1 = asymmetry_score_1(inverted_mask)
    asymmetry_2 = asymmetry_score_2(inverted_mask)
    
    # Calculate border irregularity score
    border_irregularity = border_irregularity_score(inverted_mask)

    # Calculate color asymmetry scores
    color_asymmetry_1_score = color_asymmetry_1(masked_image)
    color_asymmetry_2_score = color_asymmetry_2(masked_image)

    # Calculate texture scores
    texture_contrast_score = texture_score_contrast(gray_image)
    texture_dissimilarity_score = texture_score_dissimilarity(gray_image)
    
    # Combine all features into an array
    feature_vector = np.array([asymmetry_1, asymmetry_2, border_irregularity, 
                               color_asymmetry_1_score, color_asymmetry_2_score,
                               texture_contrast_score, texture_dissimilarity_score], dtype=np.float16)

    return feature_vector


# Feature engineering functions

def asymmetry_score_1(image):
    image = image.convert('1')
    width, height = image.size

    area_top, area_bottom, area_left, area_right = 0, 0, 0, 0
    for y in range(height):
        for x in range(width):
            pixel_value = image.getpixel((x, y))
            if pixel_value == 0:
                if y < height // 2:
                    area_top += 1
                else:
                    area_bottom += 1

                if x < width // 2:
                    area_left += 1
                else:
                    area_right += 1

    vertical_asymmetry = abs(area_top - area_bottom) / (area_top + area_bottom)
    horizontal_asymmetry = abs(area_left - area_right) / (area_left + area_right)

    average_asymmetry = (vertical_asymmetry + horizontal_asymmetry) / 2
    return average_asymmetry


def asymmetry_score_2(image, num_rotations=10):
    image = image.convert('1')
    
    image_array = np.array(image)

    center_of_mass_y, center_of_mass_x = center_of_mass(image_array)

    min_asymmetry_score = float('inf')

    for angle in np.linspace(0, 180, num_rotations, endpoint=False):

        rotated_image = rotate(image_array, angle, reshape=False)

        area_top, area_bottom, area_left, area_right = 0, 0, 0, 0
        for y in range(rotated_image.shape[0]):
            for x in range(rotated_image.shape[1]):
                pixel_value = rotated_image[y, x]
                if pixel_value == 0:
                    if y < center_of_mass_y:
                        area_top += 1
                    else:
                        area_bottom += 1

                    if x < center_of_mass_x:
                        area_left += 1
                    else:
                        area_right += 1

        vertical_asymmetry = abs(area_top - area_bottom) / (area_top + area_bottom)
        horizontal_asymmetry = abs(area_left - area_right) / (area_left + area_right)

        average_asymmetry = (vertical_asymmetry + horizontal_asymmetry) / 2

        min_asymmetry_score = min(min_asymmetry_score, average_asymmetry)

    return min_asymmetry_score


def border_irregularity_score(image):
    image = image.convert('1')

    mask = np.array(image)

    mask_closed = binary_closing(mask)

    contours = find_contours(mask_closed, 0.5)
    border = max(contours, key=len)

    perimeter = 0
    for i in range(len(border) - 1):
        perimeter += np.linalg.norm(border[i + 1] - border[i])

    area = np.sum(mask)

    irregularity_score = perimeter / area
    return irregularity_score


def color_asymmetry_1(image):

    lab_image = rgb2lab(image)

    height, width, _ = image.shape

    block_size = 3
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size

    color_avgs = []

    for i in range(num_blocks_x):
        for j in range(num_blocks_y):
            block = lab_image[j*block_size:(j+1)*block_size, i*block_size:(i+1)*block_size, :]

            if np.sum(block[:, :, 0] < 90) >= (0.75 * block_size**2):
                color_avg = np.mean(block, axis=(0, 1))
                color_avgs.append(color_avg)

    variances = np.var(color_avgs, axis=0)
    variance_score = np.mean(variances)
    
    return variance_score


def color_asymmetry_2(image):

    lab_image = rgb2lab(image)

    lab_image = resize(lab_image, (256, 256), mode='reflect', anti_aliasing=True)

    center_line = lab_image.shape[1] // 2

    left_half = lab_image[:, :center_line]
    right_half = lab_image[:, center_line:]

    right_half_flipped = np.fliplr(right_half)

    diff = np.abs(left_half - right_half_flipped)

    avg_color_diff = np.mean(diff)

    return avg_color_diff



def texture_score_contrast(image):
    if image.ndim == 3:
        image = rgb2gray(image)

    image = img_as_ubyte(image)

    glcm = graycomatrix(image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

    contrast = graycoprops(glcm, 'contrast')[0, 0]

    return contrast


def texture_score_dissimilarity(image):
    if image.ndim == 3:
        image = rgb2gray(image)

    image = img_as_ubyte(image)

    glcm = graycomatrix(image, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]

    return dissimilarity


01_process:images.py

In [2]:
import os
from os.path import exists
import pandas as pd
import numpy as np
import cv2
from PIL import Image

# Import our own file that has the feature extraction functions


# TO REINSTATE:
# from extract_features import extract_features



#-------------------
# Main script
#-------------------


#Where is the raw data
file_data = '..' + os.sep + 'data' + os.sep +'selected_images.csv'
path_image = '..' + os.sep + 'data' + os.sep + 'images' + os.sep + 'imgs_part_1'
path_mask = '..' + os.sep + 'data' + os.sep + 'shanon_masks_total'
    
#Where we will store the features
if not os.path.exists('features'):
    os.makedirs('features')
    
file_features = 'features/features.csv'

#Read meta-data into a Pandas dataframe
df = pd.read_csv(file_data)

# Extract image IDs and labels from the data. 
image_id = list(df['img_id'])
label = np.array(df['diagnostic'])

# Here you could decide to filter the data in some way (see task 0)
# For example you can have a file selected_images.csv which stores the IDs of the files you need
num_images = len(image_id)

#Make array to store features
feature_names = ["Asymmetry_Score_1", 
                 "Asymmetry_Score_2", 
                 "Border_Irregularity_Score", 
                 "Color_Asymmetry_Score_1", 
                 "Color_Asymmetry_Score_2", 
                 "Texture_Contrast_Score", 
                 "Texture_Dissimilarity_Score"]

num_features = len(feature_names)
features = np.zeros([num_images, num_features], dtype=np.float16)


#Loop through all images
for i in np.arange(num_images):
    
    # Define filenames related to this image
    image_path = path_image + os.sep + image_id[i] 
    mask_path = path_mask + os.sep + image_id[i]
    
    if exists(image_path) and exists(mask_path):
        
        # Read the image
        image = Image.open(image_path)
        mask = Image.open(mask_path)
    
        # Measure features - this does not do anything useful yet!
        x = extract_features(image, mask) 
           
        # Store in the variable we created before
        features[i,:] = x
       
        
#Save the image_id used + features to a file   
df_features = pd.DataFrame(features, columns=feature_names)     
df_features.to_csv(file_features, index=False)  

C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:29: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  image = image_obj.resize((512, 512), resample=Image.BILINEAR)
C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:30: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  mask = mask_obj.resize((512, 512), resample=Image.BILINEAR)


02_train_classifier.py

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 
import pickle

# Where are the files
file_data = '..' + os.sep + 'data' + os.sep + 'selected_images.csv'
df = pd.read_csv(file_data)
label = np.array(df['diagnostic'])

# Where did we store the features?
file_features = 'features/features.csv'
feature_names = [
    "Asymmetry_Score_1",
    "Asymmetry_Score_2",
    "Border_Irregularity_Score",
    "Color_Asymmetry_Score_1",
    "Color_Asymmetry_Score_2",
    "Texture_Contrast_Score",
    "Texture_Dissimilarity_Score"
]

# Load the features - remember the example features are not informative
df_features = pd.read_csv(file_features)

# Select only the desired features
selected_features = [
    "Asymmetry_Score_1",
    "Asymmetry_Score_2",
    "Border_Irregularity_Score",
    "Color_Asymmetry_Score_1",
    "Color_Asymmetry_Score_2"
]
x_selected = np.array(df_features[selected_features])

# Make the dataset, you can select different classes (see task 0)
x = np.array(df_features[feature_names])
y = label == 'MEL'  # now True means Melanoma, False means something else

# Prepare the train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Different classifiers to test out
classifiers = [
    KNeighborsClassifier(1),
    KNeighborsClassifier(5),
    tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ]),
    KNeighborsClassifier(5)  # 4th classifier using selected features
]
num_classifiers = len(classifiers)

acc_test = np.empty([num_classifiers])

for j, clf in enumerate(classifiers):
    if j == 3:  # the last classifier uses selected features
        x_train, x_test, y_train, y_test = train_test_split(x_selected, y, test_size=0.2, random_state=42)


    if isinstance(clf, tf.keras.Sequential):
        clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        clf.fit(x_train, y_train, epochs=10, verbose=0)
        y_pred = (clf.predict(x_test) > 0.5).flatten().astype(bool)
    else:
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
    acc_test[j] = accuracy_score(y_test, y_pred)

print('Classifier 1 test accuracy={:.3f} '.format(acc_test[0]))
print('Classifier 2 test accuracy={:.3f} '.format(acc_test[1]))
print('Classifier 3 test accuracy={:.3f} '.format(acc_test[2]))
print('Classifier 4 (selected features) test accuracy={:.3f} '.format(acc_test[3]))  # new print statement

# Let's say you now decided to use the 5-NN with selected features
classifier = KNeighborsClassifier(5)

# It will be tested on external data, so we can try to maximize the use of our available data by training on 
# ALL of the selected features and y
x_selected = np.array(df_features[selected_features])  # re-compute x_selected in case it has been changed
classifier = classifier.fit(x_selected, y)

# This is the classifier you need to save using pickle, add this to your zip file submission
filename = 'groupXY_classifier.sav'
pickle.dump(classifier, open(filename, 'wb'))


1/1 [==============================] - 0s 81ms/step
Classifier 1 test accuracy=0.464 
Classifier 2 test accuracy=0.607 
Classifier 3 test accuracy=0.607 
Classifier 4 (selected features) test accuracy=0.607 


03_evaluate.classifier.py

In [4]:
import cv2

image_path = '..\\data\\images_resized\\PAT_710_1330_243.PNG'
mask_path = '..\\data\\shanon_masks_total\\PAT_710_1330_243.PNG'

img = Image.open(image_path)
mask = Image.open(mask_path)

def classify(img, mask):
     
     selected_features_indices = [0, 1, 2, 3, 4]
   
     #Extract features (the same ones that you used for training)
     x = extract_features(img, mask)
     
     # Select only the desired features
     x_selected = x[selected_features_indices].reshape(1, -1)
     
     #Load the trained classifier
     classifier = pickle.load(open('groupXY_classifier.sav', 'rb'))
    
     #Use it on this example to predict the label AND posterior probability
     pred_label = classifier.predict(x_selected)
     pred_prob = classifier.predict_proba(x_selected)
     
     print('predicted label is ', pred_label)
     print('predicted probability is ', pred_prob)
     return pred_label, pred_prob


def classify_path(img_path, mask_path):
          
     # Load the image and mask from the path
     img = Image.open(image_path)
     mask = Image.open(mask_path)
    
     selected_features_indices = [0, 1, 2, 3, 4]
   
     #Extract features (the same ones that you used for training)
     x = extract_features(img, mask)
     
     # Select only the desired features
     x_selected = x[selected_features_indices].reshape(1, -1)
         
     
     #Load the trained classifier
     classifier = pickle.load(open('groupXY_classifier.sav', 'rb'))
    
     #Use it on this example to predict the label AND posterior probability
     pred_label = classifier.predict(x_selected)
     pred_prob = classifier.predict_proba(x_selected)
     
     
     print('predicted label is ', pred_label)
     print('predicted probability is ', pred_prob)
     return pred_label, pred_prob


classify(img, mask)
classify_path(image_path, mask_path)

# The TAs will call the function above in a loop, for external test images/masks

C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:29: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  image = image_obj.resize((512, 512), resample=Image.BILINEAR)
C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:30: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  mask = mask_obj.resize((512, 512), resample=Image.BILINEAR)


predicted label is  [False]
predicted probability is  [[0.8 0.2]]


C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:29: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  image = image_obj.resize((512, 512), resample=Image.BILINEAR)
C:\Users\jackh\AppData\Local\Temp\ipykernel_26532\1869573183.py:30: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  mask = mask_obj.resize((512, 512), resample=Image.BILINEAR)


predicted label is  [False]
predicted probability is  [[0.8 0.2]]


(array([False]), array([[0.8, 0.2]]))